# Section 4 — Data Manipulations with Pandas

Pandas is one of the most important Python libraries for **data manipulation and analysis**.  
It is especially useful in **machine learning workflows**, where raw data must first be **cleaned, transformed, and structured** before being used to train models.  

You can think of Pandas as **Excel for Python — but far more powerful and flexible**.  
Where NumPy focuses on numerical arrays, Pandas focuses on **tabular data**: rows and columns.

In this section, we will:
- Learn the basics of Pandas.
- Explore and filter data.
- Practice with small, **industry-inspired datasets**.

## 4.1 Series and DataFrames

The two core Pandas objects are:

- **Series**: a one-dimensional labeled array (like one column of a table).  
- **DataFrame**: a two-dimensional table with labeled rows and columns.

We’ll start by creating Series, then build DataFrames, and finally explore ways to inspect and understand them.

In [99]:
 # ---- Imports ----

import pandas as pd
import numpy as np
import os

# Let's also check the versions
print("Pandas version:", pd.__version__)
print("Numpy version:", np.__version__)

Pandas version: 2.2.2
Numpy version: 2.0.2


In [ ]:
# ---- Dataset Source Configuration ----
# Purpose: Make notebook cross-platform (Colab or local)
# and allow datasets to be persistent on Google Drive.

# Detect environment
try:
    import google.colab
    ON_COLAB = True
except ImportError:
    ON_COLAB = False

# Set paths based on environment and source (⚠️ Update COURSE_PATH below if your folder has a different name or location)
if ON_COLAB:
    # Mount Google Drive if using Colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    # Base folder for this course on Google Drive
    COURSE_PATH = "/content/drive/MyDrive/Industrial_ML_Course"
else:
    # Offline / local computer
    # Adjust COURSE_PATH to your local folder
    COURSE_PATH = r"D:\Industrial_ML_Course"

# Subfolders
DATASET_PATH = os.path.join(COURSE_PATH, "datasets/CWRU")
NOTEBOOK_PATH = os.path.join(COURSE_PATH, "notebooks")

# Ensure directories exist
os.makedirs(DATASET_PATH, exist_ok=True)
os.makedirs(NOTEBOOK_PATH, exist_ok=True)

print("Environment:", "Colab" if ON_COLAB else "Local")
print("Course Path:", COURSE_PATH)
print("Dataset Path:", DATASET_PATH)

In [ ]:
# ---- Creating a Series from a list ----

s = pd.Series([100, 200, 300], index=["a", "b", "c"])
print(s, "\n")

# Accessing values
print("Value at index b:", s["b"], "\n")

# Setting dtype during creation
s_float = pd.Series([1, 2, 3], dtype="float")
print(s_float)

In [ ]:
# ---- Creating DataFrame from a dictionary ----

df_dict = pd.DataFrame({
    "Date": pd.date_range("2025-10-01", periods=3, freq="D"),
    "Energy_kWh": [210, 222, 198],
    "Temperature_C": [-2, 0, 4]
})
df_dict

In [ ]:
# ---- Creating DataFrame from a NumPy array ----

data = np.array([[210, -2], [222, 0], [198, 4]])
df_array = pd.DataFrame(
    data,
    index=["2023-01-01", "2023-01-02", "2023-01-03"],
    columns=["Energy_kWh", "Temperature_C"]
)
df_array

In [ ]:
# ---- Displaying Created Dataframe ----

from IPython.display import display

# Overview
print("========== head ==========")
display(df_dict.head())      # first rows
print("\n========== info ==========")
display(df_dict.info())               # prints to stdout
print("\n========== describe ==========")
display(df_dict.describe())  # statistics
print("\n========== dtypes ==========")
display(df_dict.dtypes)      # column types

## 4.2 Selecting and Filtering Data
Data selection is one of the most common Pandas tasks.  
With small datasets, you can quickly see and pick values.  
But in **large industrial datasets** (thousands of rows, multiple column types),  
we need systematic ways to select and filter data.

Common operations include:
- Selecting **columns** (numerical, categorical, or text).  
- Selecting **rows** by label (`.loc`) or position (`.iloc`).  
- Filtering with **conditions** (e.g., energy > 280).  
- Combining multiple conditions (e.g., energy > 280 AND status = "Running").  
- Querying based on **categorical/string columns** (e.g., only Machine M2 operated by Sara).  
- Slicing rows using labels (e.g., date ranges) or integer indices.  

These tools are essential for preparing subsets of data before analysis or machine learning.

In [ ]:
# ---- Column & Row selection ----

# Column selection
display(df_dict["Energy_kWh"])

# Row by label
display(df_dict.loc[0])

# Row by integer position
display(df_dict.iloc[0])

In [ ]:
# ---- Changing the dtaframe indexes ----

df_dict.index = ["x", "y", "z"]
display(df_dict)
print("\n========== Column x ==========")
display(df_dict.loc["x"])

In [ ]:
# ---- Selecting multiple columns ----
df_dict[["Date", "Temperature_C"]]

In [ ]:
# ---- Slicing rows ----
df_dict[0:2]
df_dict.iloc[::2]  # every other rows

In [ ]:
# ---- Boolean Selection ----

# Boolean masks
df_dict[df_dict["Energy_kWh"] > 210]

# Multiple conditions
df_dict[(df_dict["Energy_kWh"] > 210) & (df_dict["Temperature_C"] < 2)]

In [ ]:
# ---- Using query for readability ----

df_dict.query("Energy_kWh > 210 and Temperature_C < 2")

In [ ]:
# ---- Label ranges (with datetime index) ----

df_idx = df_dict.set_index("Date")
df_idx.loc["2025-10-01":"2025-10-02"]

In [ ]:
# ---- Larger DataFrame with mixed data types ----

np.random.seed(42)  # reproducibility

n_rows = 2000
big_df = pd.DataFrame({
    "Date": pd.date_range("2023-01-01", periods=n_rows, freq="H"),
    "Energy_kWh": np.random.randint(150, 300, size=n_rows),        # numerical
    "Temperature_C": np.random.normal(20, 5, size=n_rows),         # numerical
    "Machine": np.random.choice(["Motor 1", "Motor 2", "Fan", "Generator", "Gear"], size=n_rows),  # categorical
    "Region": np.random.choice([" Region_A ", "region_B", " REGION_C", "Region_A", "REGION_B  "], size=n_rows),  # string
    "Status": np.random.choice(["Running", "Idle", "Fault"], size=n_rows),  # categorical
    "Operator": np.random.choice(["Radmehr", "Sara", "Rojin", "Farzad"], size=n_rows),  # string
})

big_df.head()

In [ ]:
# ---- Inspecting the big DataFrame ----

print("Shape:", big_df.shape, "\n")
print("Columns:", big_df.columns.tolist(), "\n")
big_df.info()

In [ ]:
# ---- Quick look at categorical values ----
print(big_df["Machine"].value_counts(), "\n")
print(big_df["Status"].value_counts())

In [ ]:
# ---- Example filtering on multiple types ----

subset = big_df[
    (big_df["Energy_kWh"] > 280) &
    (big_df["Status"] == "Running") &
    (big_df["Operator"] == "Sara")
]
print("Subset size:", subset.shape, "\n")
subset.head()

💡 **Industrial Example**:  
Suppose we want to detect **peak demand days** in our power grid.  
We can filter all rows where daily consumption exceeds a threshold.

## 4.3 Handling Missing and Duplicate Data

Industrial datasets often contain:
- **Missing values** → e.g., a sensor temporarily stopped recording.  
- **Duplicate rows** → e.g., the same event logged twice in a SCADA system.  

Before applying machine learning, we need to **detect, clean, and handle** these issues.  
In this subsection, we will use a **modified copy of the original dataset** to simulate such problems.

In [ ]:
# ---- Create a working copy of the original big DataFrame ----
# We will introduce missing values and duplicates in this copy

df_modified = big_df.copy()

# Introduce missing values in several columns
df_modified.loc[::50, "Energy_kWh"] = np.nan          # simulate sensor gaps
df_modified.loc[1::70, "Temperature_C"] = np.nan      # simulate sensor gaps
df_modified.loc[2::90, "Operator"] = None             # missing operator logs

# Introduce duplicate rows to simulate repeated logs
df_modified = pd.concat([df_modified, df_modified.iloc[500:510]], ignore_index=True)

df_modified.head(12)

In [ ]:
# ---- Count missing values per column ----
df_modified.isnull().sum()

In [ ]:
# ---- Percentage of missing data ----
(df_modified.isnull().mean() * 100).round(2)

In [ ]:
# ---- Option 1: Drop rows with any missing values (may discard many rows) ----

df_dropped = df_modified.dropna()
print("After dropping rows:", df_dropped.shape)

In [ ]:
# ---- Option 2: Fill missing values ----

df_filled = df_modified.copy()

# Fill numeric column with mean
df_filled["Temperature_C"].fillna(df_filled["Temperature_C"].mean(), inplace=True)

# Fill categorical/string columns with placeholders
df_filled["Operator"].fillna("Unknown", inplace=True)

df_filled.head(12)

In [ ]:
# ---- Option 3: Forward-fill (repeat last valid value, useful for time-series data) ----
df_ffill = df_modified.fillna(method="ffill")
df_ffill.head(12)

In [ ]:
# ---- Count duplicate rows ----
df_modified.duplicated().sum()

In [ ]:
# ---- Remove duplicates ----
df_no_duplicates = df_modified.drop_duplicates()
print("Original size:", df_modified.shape, "| After removing duplicates:", df_no_duplicates.shape)

💡 **Industrial Examples**
- Missing **temperature readings** can be filled with the column mean or forward-fill to maintain continuity for modeling.  
- Missing **operator logs** can use a placeholder like `"Unknown"` to avoid losing entire rows.  
- Repeated **fault events** should be removed to prevent bias in fault frequency analysis.  

## 4.4 Data Transformation

Data transformation is a key step before applying machine learning models.  
It ensures that data is in the correct format, clean, and enriched with meaningful features.

We will cover four main types of transformations:

1. **Renaming & Data Types** – making column names and data formats consistent.  
2. **Feature Engineering** – creating new features that better represent the data.  
3. **String & Text Processing** – handling textual or categorical information.  
4. **Column-wise Operations & Scaling** – applying functions, normalization, and preparing data for ML.

### 4.4.1 Column & String Transformations

Data often needs basic cleanup at both the column level and the text level.  
This includes:
- Renaming columns for clarity  
- Converting between data types (e.g., strings → datetime, floats → integers)  
- Handle datetime objects for industrial data (e.g., converting timestamps, calculating durations).
- Cleaning string/text values (removing spaces, fixing case, replacing substrings)  


In [ ]:
# ---- Rename columns ----
df_trans = big_df.copy()
df_trans = df_trans.rename(columns={"Energy_kWh": "Energy",
                                    "Temperature_C": "Temperature"})

print("Renamed columns:\n", df_trans.columns)

In [ ]:
# ---- Convert dtypes ----

# Example: ensure 'Machine' is string, 'Temperature' is float
df_trans["Machine"] = df_trans["Machine"].astype(str)
df_trans["Temperature"] = df_trans["Temperature"].astype(float)

print("\nColumn types after conversion:\n", df_trans.dtypes)

In [ ]:
# --- String/text cleaning ---
df_trans["Region"] = df_trans["Region"].str.strip().str.upper().str.replace("region_", "r")
display(df_trans)

### 4.4.2 Feature Engineering

- Add new columns derived from existing ones.  
- Example: calculate the **daily deviation from mean energy consumption**.  
- Encode categorical variables into numerical form so they can be used in ML models.

In [ ]:
# ---- Example 1: Deviation From the Mean ----

# Ensure datetime type
df_trans["Date"] = pd.to_datetime(df_trans["Date"])

# Extract just the calendar date
df_trans["Day"] = df_trans["Date"].dt.date

# Compute daily mean per calendar day
daily_mean = df_trans.groupby("Day")["Energy"].transform("mean")

# Add new feature: deviation from that mean
df_trans["Deviation"] = (
    df_trans["Energy"] - daily_mean
)

display(df_trans.head())

In [ ]:
# ---- Example 2: Encode categorical variable 'Status' into numeric ----
status_map = {"Running": 1, "Idle": 0, "Fault": -1}
df_trans["Status_Code"] = df_trans["Status"].map(status_map)
display(df_trans.head())

### 4.4.2 Column-wise Operations & Scaling

- Apply functions across columns or rows using `.apply()` and vectorized operations.  
- Normalize or scale numeric values (e.g., standardization before ML).  
- Ensure numerical stability and comparability across features.

In [ ]:
# ---- Example: Normalization ----

from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Select numeric columns
numeric_cols = ["Energy", "Temperature"]

# Min-Max Scaling to [0, 1]
scaler_minmax = MinMaxScaler()
df_trans[[col + "_MinMax" for col in numeric_cols]] = scaler_minmax.fit_transform(df_trans[numeric_cols])

# Standardization (zero mean, unit variance)
scaler_std = StandardScaler()
df_trans[[col + "_Std" for col in numeric_cols]] = scaler_std.fit_transform(df_trans[numeric_cols])

display(df_trans.head())

## 4.5 Grouping & Aggregation

In real-world industrial datasets, we often need to **summarize data**:
- How much energy did each machine consume per day?
- What was the average temperature in each region?
- How many faults occurred per status type?

Pandas provides powerful tools for these operations that generally fall into four main categories:

| **Category**            | **Function(s)**                                                                 | **Description**                                                                 |
|--------------------------|---------------------------------------------------------------------------------|---------------------------------------------------------------------------------|
| **Aggregation**          | `sum`, `mean`, `median`, `min`, `max`, `count`, `size`, `std`, `var`, `first`, `last`, `nunique`, `prod` | Reduce each group to a single scalar summary value.                             |
| **Transformation**       | `transform`, `rank`, `cumcount`, `cumsum`, `cumprod`                           | Return the same shape as the original data, but with values transformed per group. |
| **Filtering**            | `filter`                                                                       | Drop entire groups based on a condition (keep only groups satisfying a criterion). |
| **Flexible Apply / Custom** | `apply`, `agg` (or `aggregate`)                                              | Apply custom functions, or multiple aggregation functions across different columns. |

✅ **Rule of thumb**  
- Use **aggregation** for summaries.  
- Use **transform** when you need the same shape back.  
- Use **filter** to drop groups.  
- Use **apply/agg** for flexible or custom operations.

These tools are essential for preparing features and insights in Machine Learning pipelines.

In [ ]:
# ---- Reuse df_trans from earlier ----

df_group = df_trans.copy()

In [ ]:
# --- Group by machine and calculate total energy consumption ---
energy_by_machine = df_group.groupby("Machine")["Energy"].sum()
print("Total energy consumption per machine:\n", energy_by_machine.head(), "\n")

In [ ]:
# --- Group by region and calculate average temperature ---
avg_temp_by_region = df_group.groupby("Region")["Temperature"].mean()
print("Average temperature per region:\n", avg_temp_by_region, "\n")

In [ ]:
# --- Group by date (daily) and count number of faults ---
faults_per_day = df_group.groupby(df_group["Day"])["Status_Code"].apply(lambda x: (x == 1).sum())
print("Number of faults per day:\n", faults_per_day.head(), "\n")

In [ ]:
# --- Multiple aggregations with .agg() ---
machine_summary = df_group.groupby("Machine").agg({
    "Energy": ["mean", "sum", "max"],
    "Temperature": "mean"
})
print("Machine summary with multiple aggregations:\n", machine_summary.head(), "\n")

In [ ]:
# --- Group-level transformation (add column for machine-level mean energy) ---
df_group["Machine_Mean_Energy"] = df_group.groupby("Machine")["Energy"].transform("mean")
df_group[["Machine", "Energy", "Machine_Mean_Energy"]].head(10)

### 4.6 Concatenation & Merging

In real-world datasets, information often comes from multiple sources:  
- **Measurements** (e.g., sensor readings)  
- **Metadata** (e.g., machine type, location)  
- **External logs** (e.g., maintenance, failures)  

To analyze such data effectively, we need to **combine DataFrames**. Pandas provides three main approaches:  

- **Concatenation (`pd.concat`)**  
  - Stack DataFrames vertically (rows) or horizontally (columns).  
  - Useful when data has the same schema but split across files or batches.  

- **Merge (`pd.merge`)**  
  - SQL-style joins on one or more keys (`inner`, `left`, `right`, `outer`).  
  - Flexible way to combine related information across tables.  

These techniques allow building a **unified dataset** for deeper analysis.  

In [ ]:
# ---- Example 1: Concatenation (stacking DataFrames) ----

# Two small DataFrames with same columns
df_part1 = pd.DataFrame({
    "Machine": ["Motor 1", "Motor 2"],
    "Energy": [120, 150]
})

df_part2 = pd.DataFrame({
    "Machine": ["Fan", "Generator"],
    "Energy": [90, 300]
})

# Concatenate vertically (row-wise)
df_concat = pd.concat([df_part1, df_part2], ignore_index=True)
print(df_concat)

In [ ]:
# ---- Example 2: Merge (SQL-style join) ----

# Left DataFrame: machine info
df_machines = pd.DataFrame({
    "Machine": ["Motor 1", "Motor 2", "Fan", "Generator"],
    "Region": ["North", "East", "West", "South"]
})

# Right DataFrame: maintenance logs
df_maintenance = pd.DataFrame({
    "Machine": ["Motor 1", "Motor 2", "Gear"],
    "Last_Repair": ["2024-07-12", "2024-07-15", "2024-08-01"]
})

# Merge on 'Machine'
df_merged = pd.merge(df_machines, df_maintenance, on="Machine", how="outer")
print(df_merged)

## 4.7 Sorting & Ranking

In industrial datasets, it is often useful to **sort** or **rank** observations:

- Sort by numeric or categorical columns to identify top/bottom values.  
- Rank data within groups to find relative positions (e.g., top energy-consuming machines per day).  

Pandas provides:
- **`sort_values()`** → sort rows by column(s).  
- **`sort_index()`** → sort rows or columns by index.  
- **`rank()`** → assign ranks within a Series or DataFrame, with options for handling ties.  

These operations are useful for **feature engineering**, exploratory analysis, and reporting.

In [ ]:
# ---- Copy the transformed DataFrame ----

df_sort = df_trans.copy()

In [ ]:
# --- Sort by single column ---

df_sorted_energy = df_sort.sort_values("Energy", ascending=False)
print("Top 5 energy-consuming rows:\n", df_sorted_energy[["Day", "Energy", "Temperature", "Machine", "Region", "Status"]].head())

In [ ]:
# --- Sort by multiple columns (lexicographic: first by Region, then by Temperature) ---

df_sorted_multi = df_sort.sort_values(["Region", "Temperature"], ascending=[True, False])
print("Sorted by Region and Temperature:\n", df_sorted_energy[["Day", "Energy", "Temperature", "Machine", "Region", "Status"]].head(), "\n")

In [ ]:
# --- Rank within a column ---

# ascending=False → highest value gets rank 1;
# method="min" → ties share the lowest possible rank
df_sort["Energy_Rank"] = df_sort["Energy"].rank(method="min", ascending=False).astype(int)
df_sort[["Machine", "Energy", "Energy_Rank"]].head(10)

In [ ]:
# --- Rank within groups ---

df_sort["Daily_Energy_Rank"] = df_sort.groupby(df_sort["Date"].dt.date)["Energy"].rank(ascending=False, method="dense").astype(int)
df_sort[["Date", "Machine", "Energy", "Daily_Energy_Rank"]].head(10)

### 4.8 Loading & Saving Data

In industry projects, data often comes from files such as CSV, Excel, or MATLAB `.mat`.  
In this subsection, we show how to **read** these files into pandas DataFrames and how to **save** DataFrames back to files for later use.

We'll use examples based on the `big_df` structure we've created earlier, so you can experiment immediately without needing external files.

In [ ]:
# --- Save a DataFrame to CSV and Excel ---

# Save to CSV
big_df.to_csv(os.path.join(DATASET_PATH, "big_df_example.csv"), index=False)

# Save to Excel
big_df.to_excel(os.path.join(DATASET_PATH, "big_df_example.xlsx"), sheet_name="Data", index=False)

print("Saved big_df to CSV and Excel successfully!")

In [ ]:
# --- Read CSV and Excel back ---

# Read CSV
df_csv = pd.read_csv(os.path.join(DATASET_PATH, "big_df_example.csv"))
print("CSV loaded:")
print(df_csv.head())

# Read Excel
df_excel = pd.read_excel(os.path.join(DATASET_PATH, "big_df_example.xlsx"), sheet_name="Data")
print("\nExcel loaded:")
print(df_excel.head())

In [ ]:
# --- Saving and reading MATLAB .mat files ---

import scipy.io

# Convert DataFrame columns to numpy arrays
data_dict = {col: big_df[col].to_numpy() for col in big_df.columns}

# Save to .mat file
scipy.io.savemat(os.path.join(DATASET_PATH, "big_df_example.mat"), mdict=data_dict)
print("Saved big_df to MATLAB .mat file!")

# Load .mat file
mat_data = scipy.io.loadmat(os.path.join(DATASET_PATH, "big_df_example.mat"))

# Convert loaded data back to DataFrame
df_mat = pd.DataFrame({k: v.flatten() for k, v in mat_data.items() if not k.startswith("__")})
print("\nLoaded .mat data as DataFrame:")
print(df_mat.head())

### Final Exercises — Practice with Pandas

Use the `df_trans` DataFrame created in this notebook (our industrial dataset) to solve the tasks below.  
These exercises cover concepts from Sections 4.1–4.7 and will help consolidate your learning.

---

#### 1. Basic Exploration
- Display the first 10 rows and check column data types.
- Check for missing values in all columns.
- Compute basic statistics (mean, min, max) for numeric columns.

#### 2. Column Operations
- Rename `'Energy'` → `'Energy_kWh'`.
- Create a new column `'Temp_F'` converting `'Temperature'` to Fahrenheit.
- Create a flag column `'High_Energy'` for rows where `'Energy_kWh' > 400`.

#### 3. Handling Missing Data
- Introduce some NaNs in `'Temperature'` and `'Energy_kWh'` (e.g., every 60th row).
- Fill NaNs in `'Temperature'` with the column mean.
- Drop rows where `'Energy_kWh'` is NaN.

#### 4. String & Text Processing
- Clean the `'Region'` column (strip spaces, lowercase, replace `'region_'` with `'Region'`).
- Create a new column `'Machine_Type'` based on `'Machine'` (e.g., extract `'Motor'`, `'Fan'`, etc.).

#### 5. Grouping & Aggregation
- Compute total and average energy per `'Machine'`.
- Count the number of faults per `'Region'`.
- Compute standard deviation and variance of `'Temperature'` per `'Region'`.
- Rank machines by average daily energy consumption.
- Filter groups to keep only regions with more than 100 samples.

#### 6. Concatenation & Merging
- Create a small DataFrame with last maintenance dates for some machines.
- Merge it with `df_trans` on `'Machine'` using a left join.
- Concatenate a subset of `df_trans` (e.g., first 5 rows) to itself vertically.

#### 7. Sorting & Ranking
- Sort `df_trans` by `'Energy_kWh'` descending.
- Sort `df_trans` by `'Region'` and then `'Temperature'` ascending.
- Create a new column `'Energy_Rank_Global'` ranking all rows by `'Energy_kWh'`.
- Create a new column `'Energy_Rank_Daily'` ranking rows within each day.

#### 8. Loading & Saving Data
- Try modifying `big_df` (e.g., adding missing values, new columns) and save it again.
- Compare the differences in types when reading/writing CSV, Excel, and `.mat`.
- Combine this with the data cleaning and transformation tools learned in previous subsections.

#### 9. Optional: Your Choice Function
- Apply one additional GroupBy function of your choice (e.g., `nunique`, `cumsum`, `first`) to explore.
- Explain in a comment why you chose this function.

---

💡 **Hints:**  
Use pandas methods such as:
- `head()`, `info()`, `describe()`, `dtypes`  
- `rename()`, `astype()`, `apply()`, `str.strip()`, `str.lower()`, `str.replace()`  
- `isnull()`, `fillna()`, `dropna()`  
- `groupby()`, `agg()`, `transform()`, `filter()`, `rank()`  
- `merge()`, `join()`, `concat()`  
- `sort_values()`, `sort_index()`

Next → **Section 5: Data Visualization**